# Proyecto Final ML: Clustering y Semi-Supervised Learning
## Análisis de Cooperativas del Segmento 1 en Ecuador

---

**Curso:** Machine Learning

**Objetivo:** Agrupar cooperativas de ahorro y crédito según características financieras y validar coherencia de clusters contra ratings reales.

**Fechas:** Noviembre 2025

---

## 📋 Tabla de Contenidos

1. **Setup e Instalación** - Configuración inicial
2. **Parte 1: Obtención de Datos** - Web scraping y extracción
3. **Parte 2: Análisis Exploratorio (EDA)** - Exploración de datos
4. **Parte 3: Clustering No Supervisado** - K-Means, Agglomerative, DBSCAN
5. **Parte 4: Semi-Supervised Learning** - Label Propagation, Self-Training
6. **Resultados y Conclusiones** - Análisis final

## 1️⃣ SETUP E INSTALACIÓN

In [ ]:
# Setup para Google Colab (comentar si es local)
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("📱 Ejecutándose en Google Colab")
    
    # Clonar repositorio
    !git clone https://github.com/jjjulianleon/ProyectoFinalML.git
    %cd ProyectoFinalML
    
    # Instalar dependencias
    !pip install -q -r requirements.txt
    
    print("✓ Dependencias instaladas")
else:
    print("💻 Ejecutándose localmente")

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
import os
from pathlib import Path

# Configurar estilo
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
warnings.filterwarnings('ignore')

# Agregar src al path
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

# Imports de módulos locales
from etl.generate_sample_data import generate_sample_cooperativas_data
from models.clustering import ClusteringAnalyzer
from models.semi_supervised import SemiSupervisedLearner

print("✓ Imports completados")

In [ ]:
# Configuración
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Crear directorio para figuras
Path('figures').mkdir(exist_ok=True)
Path('data/processed').mkdir(parents=True, exist_ok=True)

print("✓ Configuración completada")

## 2️⃣ PARTE 1: OBTENCIÓN Y PREPARACIÓN DE DATOS

**EXTRACCIÓN AUTOMÁTICA 100%:**
- Descarga automática de PDFs desde URLs
- Extracción de texto con pdfplumber
- Procesamiento con OpenAI API (LLM)
- Generación de dataset estructurado

In [ ]:
# ⚠️  REQUISITO: EXTRACCIÓN AUTOMÁTICA 100% CON DATOS REALES
# Este notebook REQUIERE datos reales extraídos mediante web scraping con OpenAI API
# NO utiliza datos de ejemplo/prueba

print("="*70)
print("🚀 EJECUTANDO PIPELINE ETL - EXTRACCIÓN AUTOMÁTICA DE DATOS REALES")
print("="*70)
print("\n⚠️  REQUISITO IMPORTANTE:")
print("   Este análisis REQUIERE datos reales extraídos de PDFs")
print("   Se usará web scraping automático con OpenAI API")
print("   NO se usarán datos de ejemplo/prueba\n")

# Detectar si estamos en Google Colab
import sys
import os
from getpass import getpass

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("📱 Ejecutándose en Google Colab")
    print("\n🔑 CONFIGURACIÓN DE API KEY")
    print("="*70)
    print("Necesitamos tu API key de OpenAI para extraer datos de PDFs")
    print("Obtén una en: https://platform.openai.com/api-keys\n")
    print("Recomendado: Usar Google Colab Secrets")
    print("  1. Click en 🔑 (llave) en panel izquierdo")
    print("  2. Agregar secreto: OPENAI_API_KEY")
    print("  3. Pegar tu API key")
    print("="*70 + "\n")
    
    # Intentar obtener de Colab Secrets
    api_key = None
    try:
        from google.colab import userdata
        api_key = userdata.get('OPENAI_API_KEY')
        if api_key:
            print("✓ API Key obtenida de Google Colab Secrets\n")
    except Exception as e:
        print(f"⚠️  No se pudo acceder a Colab Secrets\n")
    
    # Si no está en secrets, pedir al usuario
    if not api_key:
        api_key = getpass("Ingresa tu OpenAI API Key: ")
        print()
    
    # Validar que se proporcionó API key
    if not api_key or api_key.strip() == "":
        print("❌ ERROR: API Key es requerida para extraer datos reales")
        print("   No se puede continuar sin API key")
        raise ValueError("API Key no proporcionada - Extracción de datos reales es OBLIGATORIA")
    
    os.environ['OPENAI_API_KEY'] = api_key
    os.environ['MODEL_NAME'] = 'gpt-4o-mini'
    
else:
    print("💻 Ejecutándose localmente")
    print("\n🔑 CONFIGURACIÓN DE API KEY")
    print("="*70)
    print("Necesitamos tu API key de OpenAI para extraer datos de PDFs")
    print("Obtén una en: https://platform.openai.com/api-keys\n")
    
    # Intentar obtener de .env
    from dotenv import load_dotenv
    load_dotenv(override=True)
    
    api_key = os.getenv('OPENAI_API_KEY')
    
    if not api_key:
        api_key = getpass("Ingresa tu OpenAI API Key: ")
        print()
    
    # Validar que se proporcionó API key
    if not api_key or api_key.strip() == "":
        print("❌ ERROR: API Key es requerida para extraer datos reales")
        print("   Configura OPENAI_API_KEY en .env o proporciona la key cuando se solicite")
        raise ValueError("API Key no proporcionada - Extracción de datos reales es OBLIGATORIA")
    
    os.environ['OPENAI_API_KEY'] = api_key
    os.environ['MODEL_NAME'] = 'gpt-4o-mini'

# Ejecutar pipeline ETL (OBLIGATORIO - sin fallback)
print("="*70)
print("📥 INICIANDO EXTRACCIÓN DE DATOS REALES")
print("="*70 + "\n")

try:
    from etl.run_etl_pipeline import run_etl_pipeline
    
    # Ejecutar pipeline (descarga + extracción con OpenAI)
    df = run_etl_pipeline(
        urls_file="data/cooperativas_urls.txt",
        output_csv="data/processed/cooperativas_data.csv",
        download_dir="data/raw"
    )
    
    # Validar resultados
    if df is None or df.empty:
        print("\n❌ ERROR CRÍTICO: No se extrajeron datos")
        print("   • Verifica que los URLs en data/cooperativas_urls.txt sean válidos")
        print("   • Verifica que tu API key de OpenAI sea válida")
        print("   • Verifica tu conexión a internet")
        print("   • Intenta ejecutar nuevamente")
        raise ValueError("Extracción de datos falló - No hay datos para analizar")
    
    print(f"\n✅ ÉXITO: {len(df)} muestras extraídas correctamente\n")
    print(f"📊 Distribución de Ratings:")
    print(df['rating'].value_counts().sort_index())

except Exception as e:
    print("\n❌ ERROR EN EXTRACCIÓN DE DATOS REALES:")
    print(f"   {str(e)}\n")
    print("Acciones recomendadas:")
    print("   1. Verifica que tu API key sea válida")
    print("   2. Verifica que los URLs en data/cooperativas_urls.txt sean accesibles")
    print("   3. Verifica tu conexión a internet")
    print("   4. Asegúrate de que tienes crédito en OpenAI")
    print("   5. Intenta ejecutar nuevamente\n")
    raise

print("\n" + "="*70)
print("✅ DATOS REALES CARGADOS Y LISTOS PARA ANÁLISIS")
print("="*70)

In [ ]:
# Inspeccionar datos
print("📋 Primeras filas del dataset:")
display(df.head(10))

print("\n📊 Información del dataset:")
print(df.info())

In [ ]:
# Estadísticas descriptivas
print("📈 Estadísticas Descriptivas:")
display(df.describe())

# Verificar valores faltantes
print("\n❓ Valores Faltantes:")
print(df.isnull().sum())

In [ ]:
# Guardar datos procesados
df.to_csv('data/processed/cooperativas_data.csv', index=False)
print("✓ Datos guardados en: data/processed/cooperativas_data.csv")

## 3️⃣ PARTE 2: ANÁLISIS EXPLORATORIO (EDA)

Exploración no supervisada de los datos para identificar patrones.

In [ ]:
# Seleccionar variables numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

print(f"Variables numéricas a analizar ({len(numeric_cols)}):")
for i, col in enumerate(numeric_cols, 1):
    print(f"  {i}. {col}")

In [ ]:
# Distribución por rating
fig, axes = plt.subplots(3, 4, figsize=(16, 12))
axes = axes.ravel()

for idx, col in enumerate(numeric_cols):
    has_data = False  # Track if we have any data to plot
    
    for rating in sorted(df['rating'].unique()):
        # Drop NaN values for this rating-column combination
        data = df[df['rating'] == rating][col].dropna()
        
        # Only plot if there's actual data
        if len(data) > 0:
            axes[idx].hist(data, alpha=0.5, label=f'Rating {rating}', bins=10)
            has_data = True
    
    # Set title and labels
    axes[idx].set_title(col, fontsize=10, fontweight='bold')
    axes[idx].set_xlabel('Valor')
    axes[idx].set_ylabel('Frecuencia')
    
    # Only show legend if there's data
    if has_data:
        axes[idx].legend(fontsize=8)
    else:
        axes[idx].text(0.5, 0.5, 'Sin datos disponibles', 
                      ha='center', va='center', transform=axes[idx].transAxes,
                      fontsize=10, color='red')
    
    axes[idx].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('figures/01_distribucion_por_rating.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Gráfico guardado: figures/01_distribucion_por_rating.png")

In [ ]:
# Matriz de correlación
corr_matrix = df[numeric_cols].corr()

plt.figure(figsize=(14, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación - Indicadores Financieros', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('figures/02_matriz_correlacion.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Gráfico guardado: figures/02_matriz_correlacion.png")

In [ ]:
# Análisis de correlaciones altas
print("🔗 Correlaciones más altas (excluyendo diagonal):")
corr_pairs = []

for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        corr_pairs.append({
            'variable1': corr_matrix.columns[i],
            'variable2': corr_matrix.columns[j],
            'correlacion': corr_matrix.iloc[i, j]
        })

corr_pairs_df = pd.DataFrame(corr_pairs).sort_values('correlacion', ascending=False, key=abs)
display(corr_pairs_df.head(10))

In [ ]:
# Reducción dimensional con t-SNE (o PCA como fallback)
print("🔄 Aplicando reducción dimensional para visualización...")

# IMPORTANTE: Manejar valores faltantes (NaN) ANTES de escalar
print("📋 Manejo de valores faltantes...")

# Opción 1: Eliminar filas con NaN
df_clean = df[numeric_cols].dropna()

if len(df_clean) == 0:
    print("❌ ERROR: Todas las filas contienen valores faltantes")
    print("   No hay datos para visualizar")
    raise ValueError("No hay datos completos para análisis dimensional")

if len(df_clean) < len(df):
    print(f"⚠️  Eliminadas {len(df) - len(df_clean)} filas con valores faltantes")
    print(f"   Filas restantes: {len(df_clean)}")

# Normalizar datos LIMPIOS
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_clean)

n_samples = X_scaled.shape[0]
n_features = X_scaled.shape[1]

print(f"📊 Parámetros de reducción:")
print(f"  • Número de muestras: {n_samples}")
print(f"  • Número de features: {n_features}")

# CRÍTICO: Validar que tenemos suficientes muestras para reducción dimensional
if n_samples < 2:
    print(f"\n❌ ERROR CRÍTICO: Dataset insuficiente para visualización")
    print(f"   Solo tienes {n_samples} muestra(s)")
    print(f"   Se necesitan mínimo 2 muestras para cualquier reducción dimensional")
    print(f"\nACCIONES:")
    print(f"   1. Verifica que data/cooperativas_urls.txt tenga URLs válidos")
    print(f"   2. Asegúrate de que TODOS los PDFs se descargaron exitosamente")
    print(f"   3. Verifica que la extracción con OpenAI extrae múltiples filas por PDF")
    print(f"   4. Intenta con más URLs de cooperativas")
    raise ValueError(f"Insuficientes muestras para visualización: {n_samples} < 2")

# Decidir entre t-SNE y PCA basado en tamaño del dataset
if n_samples < 10:
    print(f"\n⚠️  Dataset muy pequeño ({n_samples} < 10)")
    
    if n_samples == 2:
        print("   Solo 2 muestras - saltando visualización dimensional")
        print("   (No se puede reducir dimensionalidad con solo 2 puntos)")
        X_reduced = X_scaled  # Usar datos sin reducción
        method_name = "Original (sin reducción)"
    else:
        print("   Usando PCA en lugar de t-SNE (más estable para pocos datos)")
        
        # PCA: máximo n_components = min(n_samples, n_features)
        max_components = min(n_samples - 1, n_features)
        n_components = min(2, max_components)  # Intentar 2, pero máximo lo que permite
        
        print(f"   PCA con n_components={n_components} (max permitido: {max_components})")
        
        from sklearn.decomposition import PCA
        reducer = PCA(n_components=n_components, random_state=RANDOM_STATE)
        X_reduced = reducer.fit_transform(X_scaled)
        method_name = f"PCA (n_components={n_components})"
        
        print(f"✓ PCA completado")
        if n_components > 0:
            print(f"  Varianza explicada: {reducer.explained_variance_ratio_.sum():.1%}")

elif n_samples < 15:
    print(f"\n⚠️  Dataset pequeño ({n_samples} muestras)")
    print("   Usando t-SNE con perplexity conservador")
    
    # Para datasets muy pequeños, usar perplexity muy pequeño
    dynamic_perplexity = max(3, min(5, (n_samples - 1) // 3))
    
    print(f"  • Perplexity dinámico: {dynamic_perplexity}")
    
    try:
        tsne = TSNE(
            n_components=2, 
            random_state=RANDOM_STATE, 
            n_iter=1000, 
            perplexity=dynamic_perplexity,
            learning_rate=200,
            n_iter_without_progress=300
        )
        X_reduced = tsne.fit_transform(X_scaled)
        method_name = "t-SNE (conservador)"
        print(f"✓ t-SNE completado")
    except Exception as e:
        print(f"⚠️  t-SNE falló: {e}")
        print("   Fallback a PCA")
        
        from sklearn.decomposition import PCA
        max_components = min(n_samples - 1, n_features)
        n_components = min(2, max_components)
        
        reducer = PCA(n_components=n_components, random_state=RANDOM_STATE)
        X_reduced = reducer.fit_transform(X_scaled)
        method_name = f"PCA (fallback, n_components={n_components})"

else:
    print("✓ Dataset adecuado para t-SNE")
    
    # Para datasets normales, usar perplexity automático
    dynamic_perplexity = max(5, min(50, n_samples // 3))
    
    print(f"  • Perplexity dinámico: {dynamic_perplexity}")
    
    tsne = TSNE(
        n_components=2, 
        random_state=RANDOM_STATE, 
        n_iter=1000, 
        perplexity=dynamic_perplexity
    )
    X_reduced = tsne.fit_transform(X_scaled)
    method_name = "t-SNE"
    print(f"✓ t-SNE completado")

# Visualizar
plt.figure(figsize=(10, 8))

# Mapear ratings a colores (solo para filas sin NaN)
df_clean_with_ratings = df.loc[df_clean.index].copy()

rating_to_color = {rating: idx for idx, rating in enumerate(sorted(df_clean_with_ratings['rating'].unique()))}
colors = [rating_to_color[r] for r in df_clean_with_ratings['rating']]

scatter = plt.scatter(
    X_reduced[:, 0] if X_reduced.shape[1] > 0 else [0]*len(df_clean), 
    X_reduced[:, 1] if X_reduced.shape[1] > 1 else [0]*len(df_clean), 
    c=colors,
    cmap='viridis', 
    s=100, 
    alpha=0.6, 
    edgecolors='black', 
    linewidth=1
)

# Agregar leyenda con ratings
for rating in sorted(df_clean_with_ratings['rating'].unique()):
    mask = df_clean_with_ratings['rating'] == rating
    x_data = X_reduced[mask, 0] if X_reduced.shape[1] > 0 else [0]*mask.sum()
    y_data = X_reduced[mask, 1] if X_reduced.shape[1] > 1 else [0]*mask.sum()
    
    plt.scatter(
        x_data, y_data, 
        label=f'Rating {rating}', 
        s=100, 
        alpha=0.6,
        edgecolors='black',
        linewidth=1
    )

plt.xlabel(f'Dimensión 1' if X_reduced.shape[1] > 0 else 'Índice', fontsize=11)
plt.ylabel(f'Dimensión 2' if X_reduced.shape[1] > 1 else 'Valor', fontsize=11)
plt.title(f'Visualización con {method_name} - Cooperativas por Rating', fontsize=14, fontweight='bold')
plt.legend(loc='best')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('figures/03_tsne_visualization.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Gráfico guardado: figures/03_tsne_visualization.png")

## 4️⃣ PARTE 3: CLUSTERING NO SUPERVISADO

Aplicamos múltiples algoritmos de clustering y evaluamos su rendimiento.

In [ ]:
# Inicializar analizador de clustering
print("🤖 Inicializando analizador de clustering...\n")

# Usar datos limpios (sin NaN) - siguiendo el preprocessing de cell 16
df_clustering = df.loc[df_clean.index][numeric_cols].copy()

print(f"📊 Datos para clustering:")
print(f"  • Muestras: {len(df_clustering)}")
print(f"  • Features: {len(numeric_cols)}")
print(f"  • Sin valores faltantes: Confirmado ✓\n")

analyzer = ClusteringAnalyzer(df_clustering, random_state=RANDOM_STATE)
X_scaled = analyzer.preprocess_data()

In [ ]:
# Encontrar k óptimo para K-Means con validación para datasets pequeños
print("📊 Evaluando número óptimo de clusters (k)...\n")

n_samples = len(df)

# Calcular k_range dinámico basado en tamaño del dataset
# Regla: k debe ser < n/3 para clustering significativo
max_k_valid = max(2, n_samples // 3)

# Limitar búsqueda basada en tamaño
if n_samples < 20:
    k_range = range(2, min(max_k_valid + 1, 4))  # Máximo k=3 para datasets muy pequeños
    print(f"⚠️  Dataset pequeño ({n_samples} muestras)")
    print(f"   Limitando búsqueda a k ∈ {list(k_range)}\n")
elif n_samples < 50:
    k_range = range(2, min(max_k_valid + 1, 6))  # Máximo k=5
    print(f"⚠️  Dataset mediano ({n_samples} muestras)")
    print(f"   Limitando búsqueda a k ∈ {list(k_range)}\n")
else:
    k_range = range(2, 11)  # Búsqueda normal
    print(f"✓ Dataset adecuado ({n_samples} muestras)")
    print(f"   Búsqueda normal: k ∈ {list(k_range)}\n")

# Ejecutar búsqueda
k_results = analyzer.find_optimal_k(k_range=k_range)

# Validar resultados
if k_results.empty:
    print("❌ Error: No se pudieron calcular métricas de clustering")
    raise ValueError("find_optimal_k retornó tabla vacía")

print("✓ Búsqueda completada")
display(k_results)

In [ ]:
# Visualizar métricas de k óptimo
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Silhouette Score
axes[0].plot(k_results['k'], k_results['silhouette'], 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('Número de Clusters (k)', fontsize=11)
axes[0].set_ylabel('Silhouette Score', fontsize=11)
axes[0].set_title('Silhouette Score vs k', fontsize=12, fontweight='bold')
axes[0].grid(alpha=0.3)
axes[0].set_xticks(k_results['k'])

# Davies-Bouldin Index
axes[1].plot(k_results['k'], k_results['davies_bouldin'], 'rs-', linewidth=2, markersize=8)
axes[1].set_xlabel('Número de Clusters (k)', fontsize=11)
axes[1].set_ylabel('Davies-Bouldin Index', fontsize=11)
axes[1].set_title('Davies-Bouldin Index vs k (menor es mejor)', fontsize=12, fontweight='bold')
axes[1].grid(alpha=0.3)
axes[1].set_xticks(k_results['k'])

# Calinski-Harabasz Index
axes[2].plot(k_results['k'], k_results['calinski_harabasz'], 'gs-', linewidth=2, markersize=8)
axes[2].set_xlabel('Número de Clusters (k)', fontsize=11)
axes[2].set_ylabel('Calinski-Harabasz Index', fontsize=11)
axes[2].set_title('Calinski-Harabasz Index vs k', fontsize=12, fontweight='bold')
axes[2].grid(alpha=0.3)
axes[2].set_xticks(k_results['k'])

plt.tight_layout()
plt.savefig('figures/04_elbow_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Gráfico guardado: figures/04_elbow_analysis.png")

In [ ]:
# Seleccionar k óptimo (basado en Silhouette Score)
optimal_k = k_results.loc[k_results['silhouette'].idxmax(), 'k'].astype(int)
print(f"✓ k óptimo seleccionado: {optimal_k}")
print(f"  Silhouette Score: {k_results.loc[k_results['k'] == optimal_k, 'silhouette'].values[0]:.3f}")

In [ ]:
# Aplicar K-Means
print(f"\n{'='*50}")
kmeans_labels, kmeans_metrics = analyzer.kmeans_clustering(n_clusters=optimal_k)
print(f"{'='*50}")

In [ ]:
# Aplicar Agglomerative Clustering
print(f"\n{'='*50}")
agg_labels, agg_metrics = analyzer.agglomerative_clustering(n_clusters=optimal_k)
print(f"{'='*50}")

In [ ]:
# Aplicar DBSCAN
print(f"\n{'='*50}")
dbscan_labels, dbscan_metrics = analyzer.dbscan_clustering(eps=0.8, min_samples=4)
print(f"{'='*50}")

In [ ]:
# Resumen de métricas de clustering
print("\n📊 RESUMEN DE MÉTRICAS DE CLUSTERING\n")
clustering_summary = analyzer.get_summary()
display(clustering_summary)

# Guardar resumen
clustering_summary.to_csv('data/processed/clustering_metrics.csv', index=False)
print("\n✓ Resumen guardado: data/processed/clustering_metrics.csv")

In [ ]:
# Visualizar clusters en t-SNE/PCA
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Usar la reducción dimensional realizada en cell 16
X_reduced_data = X_reduced if X_reduced.shape[1] >= 2 else np.column_stack([X_reduced[:, 0], np.zeros(X_reduced.shape[0])])

# K-Means
scatter1 = axes[0].scatter(X_reduced_data[:, 0], X_reduced_data[:, 1], c=kmeans_labels,
                           cmap='viridis', s=100, alpha=0.6, edgecolors='black', linewidth=1)
axes[0].set_title(f'K-Means (k={optimal_k})', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Dimensión 1')
axes[0].set_ylabel('Dimensión 2')
plt.colorbar(scatter1, ax=axes[0], label='Cluster')
axes[0].grid(alpha=0.3)

# Agglomerative
scatter2 = axes[1].scatter(X_reduced_data[:, 0], X_reduced_data[:, 1], c=agg_labels,
                           cmap='plasma', s=100, alpha=0.6, edgecolors='black', linewidth=1)
axes[1].set_title(f'Agglomerative Clustering (k={optimal_k})', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Dimensión 1')
axes[1].set_ylabel('Dimensión 2')
plt.colorbar(scatter2, ax=axes[1], label='Cluster')
axes[1].grid(alpha=0.3)

# DBSCAN
scatter3 = axes[2].scatter(X_reduced_data[:, 0], X_reduced_data[:, 1], c=dbscan_labels,
                           cmap='cool', s=100, alpha=0.6, edgecolors='black', linewidth=1)
axes[2].set_title(f'DBSCAN', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Dimensión 1')
axes[2].set_ylabel('Dimensión 2')
plt.colorbar(scatter3, ax=axes[2], label='Cluster')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('figures/05_clustering_results_tsne.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Gráfico guardado: figures/05_clustering_results_tsne.png")

In [ ]:
# Comparación con ratings reales
print("\n📊 COMPARACIÓN CON RATINGS REALES\n")

# Codificar ratings a números (solo para las filas limpias)
df_for_comparison = df.loc[df_clean.index].copy()

le = LabelEncoder()
ratings_encoded = le.fit_transform(df_for_comparison['rating'])

comparison = analyzer.compare_with_ratings(ratings_encoded)

In [ ]:
# Matriz de confusión - K-Means vs Ratings
from sklearn.metrics import confusion_matrix

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

algorithms = ['K-Means', 'Agglomerative', 'DBSCAN']
labels_list = [kmeans_labels, agg_labels, dbscan_labels]

for idx, (algo_name, labels) in enumerate(zip(algorithms, labels_list)):
    cm = confusion_matrix(ratings_encoded, labels)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx], cbar=False)
    axes[idx].set_title(f'{algo_name}', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Rating Real')
    axes[idx].set_xlabel('Cluster Predicho')

plt.tight_layout()
plt.savefig('figures/06_confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Gráfico guardado: figures/06_confusion_matrices.png")

## 5️⃣ PARTE 4: SEMI-SUPERVISED LEARNING

Comparamos diferentes enfoques variando el ratio de datos etiquetados.

In [ ]:
# Inicializar learner semi-supervisado
print("🤖 Inicializando Semi-Supervised Learner...\n")

# Usar datos limpios (sin NaN) - siguiendo el preprocessing de cell 16
df_semi = df.loc[df_clean.index][numeric_cols + ['rating']].copy()

print(f"📊 Datos para semi-supervised learning:")
print(f"  • Muestras: {len(df_semi)}")
print(f"  • Features: {len(numeric_cols)}")
print(f"  • Target: rating")
print(f"  • Sin valores faltantes: Confirmado ✓\n")

semi_learner = SemiSupervisedLearner(df_semi, 
                                      target_column='rating',
                                      random_state=RANDOM_STATE)
X_semi, y_semi = semi_learner.preprocess_data()

In [ ]:
# Baseline supervisado
print("▶ Entrenando BASELINE SUPERVISADO\n")
baseline = semi_learner.supervised_baseline()

In [ ]:
# Comparación variando ratios
print("▶ Evaluando Semi-Supervised Learning con diferentes ratios\n")

ratios = [0.1, 0.2, 0.3, 0.5, 0.7]
results_df = semi_learner.compare_ratios(ratios=ratios)

print("\n✓ Evaluación completada")
display(results_df)

In [ ]:
# Guardar resultados semi-supervised
results_df.to_csv('data/processed/semi_supervised_results.csv', index=False)
print("✓ Resultados guardados: data/processed/semi_supervised_results.csv")

In [ ]:
# Visualizar comparación de métodos
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

metrics = ['accuracy', 'precision', 'recall', 'f1_score']
colors = {'Supervised Baseline': 'red', 'Label Propagation': 'blue', 'Self-Training': 'green'}

for idx, metric in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]
    
    # Graficar baseline
    baseline_value = results_df[results_df['method'] == 'Supervised Baseline'][metric].values[0]
    ax.axhline(y=baseline_value, color='red', linestyle='--', linewidth=2, label='Supervised Baseline')
    
    # Graficar Label Propagation
    lp_data = results_df[results_df['method'] == 'Label Propagation']
    ax.plot(lp_data['labeled_ratio'] * 100, lp_data[metric], 'bo-', linewidth=2, 
            markersize=8, label='Label Propagation')
    
    # Graficar Self-Training
    st_data = results_df[results_df['method'] == 'Self-Training']
    ax.plot(st_data['labeled_ratio'] * 100, st_data[metric], 'gs-', linewidth=2, 
            markersize=8, label='Self-Training')
    
    ax.set_xlabel('Porcentaje de Datos Etiquetados (%)', fontsize=11)
    ax.set_ylabel(metric.replace('_', ' ').title(), fontsize=11)
    ax.set_title(f'{metric.replace("_", " ").title()} vs Ratio de Labels', fontsize=12, fontweight='bold')
    ax.grid(alpha=0.3)
    ax.legend()
    ax.set_ylim([0, 1.05])

plt.tight_layout()
plt.savefig('figures/07_semi_supervised_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Gráfico guardado: figures/07_semi_supervised_comparison.png")

## 6️⃣ RESULTADOS Y CONCLUSIONES

In [ ]:
print("="*70)
print("RESUMEN DE RESULTADOS - CLUSTERING")
print("="*70)
print(f"\n✓ Número óptimo de clusters: {optimal_k}")
print(f"\n📊 Métricas por algoritmo:")
print(clustering_summary.to_string(index=False))
print("\n" + "="*70)
print("INTERPRETACIÓN:")
print("="*70)
print("""
1. SILHOUETTE SCORE (rango: -1 a 1)
   - Mide la similitud de un objeto con su cluster vs otros clusters
   - Valores más altos indican mejor separación
   - Interpretación: > 0.5 (bueno), > 0.7 (excelente)

2. DAVIES-BOULDIN INDEX (menor es mejor)
   - Razón promedio de similitud intra-cluster vs inter-cluster
   - Valores más bajos indican clusters mejor definidos
   - Interpretación: < 1.0 (bueno), < 0.5 (excelente)

3. COMPARACIÓN CON RATINGS REALES
   - Adjusted Rand Index mide acuerdo entre clustering y ratings
   - Rango: -1 a 1 (1 = acuerdo perfecto, 0 = acuerdo aleatorio)
""")

In [ ]:
print("="*70)
print("RESUMEN DE RESULTADOS - SEMI-SUPERVISED LEARNING")
print("="*70)
print(results_df.to_string(index=False))
print("\n" + "="*70)
print("INTERPRETACIÓN:")
print("="*70)
print("""
1. ACCURACY
   - Proporción de predicciones correctas
   - Métrica general de rendimiento

2. PRECISION
   - Proporción de predicciones positivas que fueron correctas
   - Importante cuando el costo de falsos positivos es alto

3. RECALL
   - Proporción de casos positivos que fueron identificados
   - Importante cuando el costo de falsos negativos es alto

4. F1-SCORE
   - Media armónica entre Precision y Recall
   - Métrica equilibrada para clasificación desbalanceada

5. RATIO DE LABELS
   - Proporción de datos etiquetados usados en entrenamiento
   - Medir el impacto de tener menos datos etiquetados
""")

In [ ]:
# Análisis de clusters vs ratings
print("\n" + "="*70)
print("ANÁLISIS DETALLADO: CLUSTERS K-MEANS vs RATINGS")
print("="*70)

# Crear DataFrame con resultados (usar solo filas limpias)
df_clustered = df.loc[df_clean.index].copy()
df_clustered['cluster_kmeans'] = kmeans_labels
df_clustered['cluster_agg'] = agg_labels
df_clustered['cluster_dbscan'] = dbscan_labels

print("\n📊 Distribución de ratings por cluster K-Means:")
crosstab = pd.crosstab(df_clustered['rating'], df_clustered['cluster_kmeans'], margins=True)
print(crosstab)

print("\n💡 Observaciones:")
for cluster in range(optimal_k):
    cluster_data = df_clustered[df_clustered['cluster_kmeans'] == cluster]
    rating_dist = cluster_data['rating'].value_counts()
    print(f"  Cluster {cluster}: {len(cluster_data)} cooperativas")
    print(f"    Distribución de ratings: {dict(rating_dist)}")

In [ ]:
# Conclusiones finales
print("\n" + "="*70)
print("CONCLUSIONES Y RECOMENDACIONES")
print("="*70)
print("""
🎯 HALLAZGOS PRINCIPALES:

1. CLUSTERING NO SUPERVISADO:
   • Se identificaron patrones naturales en los datos financieros
   • El número óptimo de clusters fue determinado mediante Silhouette Score
   • K-Means proporciona una buena separación de cooperativas
   • Los clusters muestran cierta coherencia con los ratings reales

2. COMPARACIÓN CON RATINGS REALES:
   • Existe una relación parcial entre clusters y ratings
   • Algunos ratings se distribuyen en múltiples clusters
   • Sugiere que los indicadores financieros capturan matices no reflejados en ratings simples

3. SEMI-SUPERVISED LEARNING:
   • Label Propagation muestra mejor rendimiento con menos datos etiquetados
   • Self-Training es más inestable en ratios bajos
   • Ambos métodos se acercan al baseline supervisado con ~50% de datos etiquetados

📌 RECOMENDACIONES:

   1. Para clasificación de nuevas cooperativas:
      → Usar modelo supervisado con todos los datos disponibles
      → Si hay nuevas cooperativas sin etiquetar, aplicar Label Propagation

   2. Para segmentación de cooperativas:
      → K-Means proporciona grupos interpretables
      → Validar grupos con expertos en finanzas

   3. Mejoras futuras:
      → Incluir más indicadores financieros
      → Validación cruzada temporal (datos históricos)
      → Análisis de estabilidad de clusters
      → Investigar por qué algunos ratings se distribuyen en múltiples clusters
""")

In [ ]:
# Guardar resultados finales
print("\n" + "="*70)
print("GUARDANDO RESULTADOS FINALES")
print("="*70)

# Guardar datos clustered
df_clustered.to_csv('data/processed/cooperativas_clustered.csv', index=False)
print("✓ Datos clustered guardados")

# Guardar métricas
clustering_summary.to_csv('data/processed/clustering_metrics.csv', index=False)
results_df.to_csv('data/processed/semi_supervised_results.csv', index=False)
print("✓ Métricas guardadas")

print("\n✅ Análisis completado exitosamente")
print("\n📁 Archivos generados:")
print("  • data/processed/cooperativas_data.csv")
print("  • data/processed/cooperativas_clustered.csv")
print("  • data/processed/clustering_metrics.csv")
print("  • data/processed/semi_supervised_results.csv")
print("  • figures/01_distribucion_por_rating.png")
print("  • figures/02_matriz_correlacion.png")
print("  • figures/03_tsne_visualization.png")
print("  • figures/04_elbow_analysis.png")
print("  • figures/05_clustering_results_tsne.png")
print("  • figures/06_confusion_matrices.png")
print("  • figures/07_semi_supervised_comparison.png")

## 📚 Referencias y Metodología

### Fuentes Teóricas

1. **Clustering No Supervisado:**
   - Lloyd, S. (1982). Least squares quantization in PCM. IEEE Transactions on Information Theory
   - Rousseeuw, P. J. (1987). Silhouettes: a graphical aid to the interpretation of cluster analysis
   - Davies, D. L., & Bouldin, D. W. (1979). A cluster separation measure

2. **Semi-Supervised Learning:**
   - Zhou, D., Bousquet, O., Lal, T. N., Weston, J., & Schölkopf, B. (2004). Learning with local and global consistency
   - Rosenberg, D., Hebert, M., & Schneiderman, H. (2005). Semi-supervised self-training of object detection models

3. **Visualización:**
   - van der Maaten, L., & Hinton, G. (2008). Visualizing Data using t-SNE

### Indicadores Financieros

Referencia: Superintendencia de Economía Popular y Solidaria (SEPS)
- https://www.seps.gob.ec
- ASIS: Asociación de Supervisores de Instituciones de Seguros
- https://www.asis.fin.ec

---

**Notebook generado:** Noviembre 2025

**Próximos pasos:**
1. Obtener datos reales de cooperativas (archivos PDF)
2. Validar resultados con expertos en finanzas
3. Realizar análisis temporal de estabilidad de clusters
4. Investigar casos discrepantes